https://www.yeastgenome.org/

https://www.cse.msu.edu/~cse835/Papers/Graph_connectivity_revised.pdf => connexite

Maslov et al., 2013, Segundo et al., 2011, Tomita and Kameda, 2007 => Ensemble stable


isthme, trouver les cycles et circonférence, sommet-connexite, centralité, nb chromatique, contraction plutôt que suppression, partition, dureté, force, stable, clique, noyau, dominants


In [4]:
import pandas as pd
import requests
import networkx

In [5]:
def parcours_profondeur(graphe: dict, sommet: int) -> list:
    """
        Algorithme de parcours en profondeur d'un graphe.
        
        Variables:
            - graphe (dict): graphe sous la forme de dictionnaire {sommet: {voisin1, voisin2, ..., voisinN}}.
            - sommet (int): correspond à la racine de l'aborescence de parcours en profondeur.
        
        Sorties:
            - pere (list): correspon à l'arborescence avec pere[x] est le père du sommet x dans le parcous en profondeur.
            
        Complexité:
            - O(n)
    """
    taille = max(graphe.keys())
    
    etat = [None for _ in range(taille)]
    pere = [None for _ in range(taille)]
    dernier_pred = [None for _ in range(taille)]
    
    nb_vu = 1
    
    pile = [sommet]
    
    while nb_vu <= taille:
        
        if nb_vu not in graphe.keys():
            
            nb_vu += 1
        
        elif etat[nb_vu - 1] is not None:
            
            nb_vu += 1
            
        elif pile == []:
            
            pile.append(nb_vu)
            
        while pile != []:
            
            suivant = pile.pop()
            
            if etat[suivant - 1] is None:
                
                etat[suivant - 1] = 0
                pere[suivant - 1] = dernier_pred[suivant - 1]
                
                for successeur in graphe[suivant]:
                    
                    pile.append(successeur)
                    dernier_pred[successeur - 1] = suivant
    return pere


def excentricite_sommet(graphe: dict, sommet: int) -> int:
    """
        Algorithme de calcul d'excentricité d'un sommet dans un graphe. Aussi la hauteur de l'aborescence d'un parcours
        en largeur du graphe à partir du sommet.
        
        Variables:
            - graphe (dict): graphe sous la forme de dictionnaire {sommet: {voisin1, voisin2, ..., voisinN}}.
            - sommet (int): correspond à la racine de l'aborescence de parcours en profondeur.
        
        Sorties:
            - max(hauteur) (int): correspond à l'excentricité du sommet.
            
        Complexité:
            - O(n)
    """
    taille = max(graphe.keys())
    
    etat = [None for _ in range(taille)]
    hauteur = [0 for _ in range(taille)]
    
    file = [sommet]
    etat[sommet - 1] = 0
    
    nb_vu = 1
    
    while nb_vu <= taille:
        
        if nb_vu not in graphe.keys():
            
            nb_vu += 1
        
        elif etat[nb_vu - 1] is not None:
            
            nb_vu += 1
            
        elif file == []:
            
            file.append(nb_vu)
            etat[nb_vu - 1] = 0
            
        while file != []:
            
            suivant = file[0]
            file = file[1:] if len(file) > 1 else []
            
            for successeur in graphe[suivant]:
                
                if etat[successeur - 1] is None:
                    
                    file.append(successeur)
                    etat[successeur - 1] = 0
                    hauteur[successeur - 1] = hauteur[suivant - 1] + 1
                    
    return max(hauteur)


def composantes_connexes(graphe: dict) -> list:
    """
        Algorithme de calcul des composantes connexes d'un graphe par parcours en profondeur, à chaque fois qu'un sommet n'a
        pas était visité alors que la pile est vide, compose connexe + 1.
        L'algorithme renvoie de plus les composantes connexes sous forme de liste avec un dictionnaire pour représenter chaque
        composante connexe.
        
        Variables:
            - graphe (dict): graphe sous la forme de dictionnaire {sommet: {voisin1, voisin2, ..., voisinN}}.
        
        Sorties: 
            - composante (list): Les composantes connexes du graphe.
            
        Complexité:
            - O(n).
    """
    taille = max(graphe.keys())
    
    etat = [None for _ in range(taille)]
    pere = [None for _ in range(taille)]
    dernier_pred = [None for _ in range(taille)]
    composante = []
    
    nb_vu = 1
    sous_graphe_connexe = {1: set()}
    
    pile = [1]
    
    while nb_vu <= taille:
        
        if nb_vu not in graphe.keys():
            
            nb_vu += 1
        
        elif etat[nb_vu - 1] is not None:
            
            nb_vu += 1
            
        elif pile == []:
            
            composante.append(sous_graphe_connexe)
            pile.append(nb_vu)
            sous_graphe_connexe = {nb_vu: set()}
            
        while pile != []:
            
            suivant = pile.pop()
            
            if etat[suivant - 1] is None:
                
                sous_graphe_connexe[suivant] = set()
                etat[suivant - 1] = 0
                pere[suivant - 1] = dernier_pred[suivant - 1]
                
                for successeur in graphe[suivant]:
                    
                    sous_graphe_connexe[successeur] = set() if etat[successeur - 1] is None else sous_graphe_connexe[successeur]
                    sous_graphe_connexe[suivant].add(successeur)
                    sous_graphe_connexe[successeur].add(suivant)
                    pile.append(successeur)
                    dernier_pred[successeur - 1] = suivant
                    
    composante.append(sous_graphe_connexe)
    
    return composante


def sommets_isoles(graphe: dict) -> set:
    """
        Algorithme qui renvoie les sommets isolées d'un graphe, ou les sommets de degré = 0.
        
        Variables:
            - graphe (dict): graphe sous la forme de dictionnaire {sommet: {voisin1, voisin2, ..., voisinN}}.
            
        Sorties:
            - isoles (set): ensemble de sommets isolées i.e de degré nul.
            
        Complexité:
            - O(n)
    """
    isoles = set()
    
    for v1, voisins in graphe.items():
        
        if len(voisins) == 0:
            
            isoles = isoles.union({v1})
    
    return isoles


def cycle(graphe: dict) -> bool:
    """
        Algorithme qui réalise un parcours en profondeur, si rencontre deux fois un sommet alors il renvoie vrai 
        si le graphe possède un cycle, faux sinon.
        
        Variables:
            - graphe (dict): graphe sous la forme de dictionnaire {sommet: {voisin1, voisin2, ..., voisinN}}.
        
        Sorties:
            - cyclique (bool): Vrai si le graphe possède un cycle, faux sinon.
            
        Complexité:
            - O(n)
    """
    cyclique = False
    
    taille = len(graphe.keys())
    
    etat = [None for _ in range(taille)]
    dernier_pred = [None for _ in range(taille)]
    
    nb_vu = 1
    
    pile = [list(graphe.keys())[0]]
    
    while nb_vu <= taille:
        
        if nb_vu not in graphe.keys():
            
            nb_vu += 1
        
        elif etat[nb_vu - 1] is not None:
            
            nb_vu += 1
            
        elif pile == []:
            
            pile.append(nb_vu)
            cc += 1
            
        while pile != []:
            
            suivant = pile.pop()
            
            if etat[suivant - 1] is None:
                
                etat[suivant - 1] = 0
                
                for successeur in graphe[suivant]:
                    
                    if (etat[successeur - 1] is not None) and (dernier_pred[suivant - 1] != successeur):
                        
                        cyclique = True
                        break
                        
                    pile.append(successeur)
                    dernier_pred[successeur - 1] = suivant
    
                if cyclique: break
            if cyclique: break
        if cyclique: break
    
    return cyclique


def points_articulations(graphe: dict) -> set:
    """
        Algorithme qui renvoie l'ensemble des points d'articulations d'un graphe, par un parcours en profondeur si la racine
        possède N fils avec N > 1 alors point d'articulation.
        
        Variables:
            - graphe (dict): graphe sous la forme de dictionnaire {sommet: {voisin1, voisin2, ..., voisinN}}.
        
        Sorties:
            - points (set): L'ensemble des points d'articulations d'un graphe.
            
        Complexité:
            - O(n^2)
    """
    points = set()
    
    for s in graphe:
        points = points.union({s}) if parcours_profondeur(graphe, s).count(s) > 1 else points
    
    return points


articulation = lambda graphe, s: True if parcours_profondeur(graphe, s).count(s) > 1 else False


def supprimer_sommet(graphe: dict, sommet: int) -> dict:
    """
        Algorithme permettant la création d'un sous-graphe induit avec G' = (V', E') et V - {sommet}.
        
        Variables:
            - graphe (dict): graphe sous la forme de dictionnaire {sommet: {voisin1, voisin2, ..., voisinN}}.
            - sommet (int): le sommet a supprimé.
        
        Sorties:
            - nouveau_graphe (dict): correspond au sous-graphe induit.
        
        Complexité:
            - O(|V|+|E|)
    """
    nouveau_graphe = graphe.copy()
    
    for s in nouveau_graphe.keys():
        nouveau_graphe[s] = nouveau_graphe[s].difference({sommet})
    del nouveau_graphe[sommet]
    
    return nouveau_graphe

def graphe_complementaire(graphe: dict) -> dict:
    """
        Algorithme qui permet de calculer le graphe complémentaire de G.
        
        Variables:
            - graphe (dict): graphe sous la forme de dictionnaire {sommet: {voisin1, voisin2, ..., voisinN}}.
        
        Sorties:
            - complementaire (dict): graphe complémentaire du graphe d'entrée
        
        Complexité:
            - O(?)
    """
    complementaire = {sommet: set(graphe.keys()) for sommet in graphe.keys()}
    
    for s, voisins in graphe.items():
        
        complementaire[s] = complementaire[s].difference(graphe[s].union({s}))

    return complementaire

    
def centres(graphe: dict) -> tuple:
    """
        Algorithme calculant l'ensemble des centres d'un graphe par le calcul de l'excentricité minimale.
        
        Variables:
            - graphe (dict): graphe sous la forme de dictionnaire {sommet: {voisin1, voisin2, ..., voisinN}}.
        
        Sorties:
            - (tuple): Renvoie (x, y) avec x l'ensemble des centres et y leurs excentricités.
            
        Complexité:
            - O(n^2)
    """
    excent = {sommet: 0 for sommet in graphe.keys()}
    
    for sommet in graphe:
        excent[sommet] = excentricite_sommet(graphe, sommet)
    
    return set(sommet for sommet in graphe if excent[sommet] == min(excent.values())), min(excent.values())


def diametre(graphe: dict) -> int:
    
    excent = {sommet: 0 for sommet in graphe.keys()}
    
    for sommet in graphe:
        excent[sommet] = excentricite_sommet(graphe, sommet)
    
    return set(sommet for sommet in graphe if excent[sommet] == max(excent.values())), max(excent.values())


densite = lambda graphe: sum(list(map(len, list(graphe.values()))))/(len(graphe.keys())*(len(graphe.keys()) - 1))


def sommets_couvrants(graphe: dict) -> set:
    
    """Chvatal algorithm à la place"""
    
    """
        Heuristique de calcul des sommets couvrants (vertex cover).
        
        Variables:
            - graphe (dict): graphe sous la forme de dictionnaire {sommet: {voisin1, voisin2, ..., voisinN}}.
        
        Sorties:
            - dom (set): l'ensemble (dont on espère minimum) des sommets couvrants l'ensemble des arêtes.
            
        Complexité:
            - O(n^2)
    """
    
    deg = {sommet: len(voisins) for sommet, voisins in graphe.items()}
    
    def procedure(graphe: dict, degres: dict, dom: set) -> set:
        
        if len(degres) == 0 or max(degres.values()) == 0:
            
            return dom
        
        nouveau = max(degres, key=lambda k: degres[k])
        
        dom = dom.union({nouveau})
        
        for sommet in degres:
            
            if nouveau in graphe[sommet]:
                
                degres[sommet] -= 1
                
        del degres[nouveau]
        
        return procedure(graphe, degres, dom)
    
    return procedure(graphe, deg, set())
    

def contraction(graphe: dict, s1: int, s2: int) -> dict:
    """
        Algorithme de contraction de deux sommets s1, s2 en un sommet appelé s1/s2.
        
        Variable:
            - graphe (dict): graphe sous la forme de dictionnaire {sommet: {voisin1, voisin2, ..., voisinN}}.
            - s1 (int): le sommet n°1 a fusionné.
            - s2 (int): le sommet n°2 a fusionné.
        
        Sorties:
            - nouveau_graphe (dict): le graphe contracté.
        
        Complexité:
            - O(?)
    """
    nouveau_graphe = graphe.copy()
    
    nouveau_graphe[f"{s1}/{s2}"] = nouveau_graphe[s1].union(graphe[s2])
    
    for sommet, voisins in graphe.items():
        if s1 in voisins or s2 in voisins:
            
            nouveau_graphe[sommet] = (nouveau_graphe[sommet].difference({s1, s2})).union({f"{s1}/{s2}"})
    
    del nouveau_graphe[s1]
    del nouveau_graphe[s2]
    
    return nouveau_graphe

def contraction_pondere(s1, s2, graphe):
    """
        Algorithme de contraction de deux sommets s1, s2 en un sommet appelé s1/s2 et pour toute arêtes incidentes à 
        s1 ou s2 on a w(s1/s2, x) = somme(w(s1, x), w(s2, x)).
        
        Variables:
            - graphe (dict): graphe sous la forme de dictionnaire {sommet: {(voisin1, poids1), (voisin2, poids2), ..., (voisinN, poidsN)}}.
            - s1 (int): le sommet n°1 a fusionné.
            - s2 (int): le sommet n°2 a fusionné.
        
        Sorties:
            - nouveau_graphe (dict): le graphe contracté.
        
        Complexité:
            - O(?)
    """
    sf = f"{s1}/{s2}"
    nouveau_graphe = graphe.copy()

    for sommet in graphe:
        nouveau_graphe[sommet] = {arete for arete in graphe[sommet] if arete[0] not in {s1, s2}}
    del nouveau_graphe[s1], nouveau_graphe[s2]

    for sommet in nouveau_graphe:
        nouveau_graphe[sommet] = nouveau_graphe[sommet].union({(sf, sum(list(map(lambda arete: arete[1], {art for art in graphe[sommet] if art[0] in {s1, s2}}))))})

    nouveau_graphe[sf] = set()    
        
    for sommet in nouveau_graphe:
        nouveau_graphe[sf] = nouveau_graphe[sf].union({(sommet, arete[1]) for arete in nouveau_graphe[sommet] if arete[0] == sf}) 

    return nouveau_graphe


def stoer_wagner(graphe: dict) -> int:
    """
        Algorithme de Stoer-Wagner afin de trouver la coupe minimale d'un graphe i.e le k pour k-arêtes-connexes.
        
        Variables:
            - graphe (dict): graphe sous la forme de dictionnaire {sommet: {voisin1, voisin2, ..., voisinN}}.
        
        Sorties:
            - (int): La coupe minimale du graphe.
        
        Complexité:
            - O(n^2)
    """
    nouveau_graphe = {sommet: {(voisin, 1) for voisin in voisins} for sommet, voisins in graphe.items()}
    
    coupe = set()
    
    while len(nouveau_graphe) > 2:
        
        graphe_poids = nouveau_graphe.copy()
        ensemble = list(sommet for sommet in nouveau_graphe if "/" in str(sommet))
        ensemble = 1 if ensemble == [] else ensemble[0]
        cpt = 0
        sommets = []
        
        while cpt < len(graphe_poids):
            
            s1, s2, w = None, None, -1
                
            v1 = ensemble
            v2, maximum_local = list(graphe_poids[ensemble])[0]

            for arete in graphe_poids[ensemble]:
                if arete[1] > maximum_local:
                    v2, maximum_local = arete

            if maximum_local > w:
                s1, s2, w = v1, v2, maximum_local
                    
            ensemble = f"{s1}/{s2}"
            sommets.append(s2)
            
            graphe_poids = contraction_pondere(s1, s2, graphe_poids)
            
            cpt += 1
            
        s1, s2 = sommets[-2:]
        
        s1 = str(s1) if "/" in str(s1) else int(s1)
        s2 = str(s2) if "/" in str(s2) else int(s2)
        
        nouveau_graphe = contraction_pondere(s1, s2, nouveau_graphe)
        coupe = coupe.union({w})
    
    coupe = coupe.union({max({poids for aretes in nouveau_graphe.values() for _, poids in aretes})})
    
    return min(coupe)
        

In [6]:
proteome = pd.read_csv("Saccharomyces cerevisiae/transcriptome.tsv", sep="\t")
proteome = proteome[proteome["Gene Type"] == "protein-coding"]
proteome

,Accession,Begin,End,Chromosome,Orientation,Name,Symbol,Gene ID,Gene Type,Transcripts accession,Protein accession,Protein length,Locus tag
0,NC_001133.9,1807,2169,I,minus,seripauperin PAU8,PAU8,851229,protein-coding,NM_001180043.1,NP_009332.1,120.0,YAL068C
1,NC_001133.9,2480,2707,I,plus,uncharacterized protein,NaN,1466426,protein-coding,NM_001184582.1,NP_878038.1,75.0,YAL067W-A
2,NC_001133.9,7235,9016,I,minus,putative permease SEO1,SEO1,851230,protein-coding,NM_001178208.1,NP_009333.1,593.0,YAL067C
3,NC_001133.9,11565,11951,I,minus,uncharacterized protein,NaN,851232,protein-coding,NM_001179897.1,NP_009335.1,128.0,YAL065C
4,NC_001133.9,12046,12426,I,plus,uncharacterized protein,NaN,851233,protein-coding,NM_001180042.1,NP_009336.1,126.0,YAL064W-B
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6443,NC_001224.1,48901,50097,MT,plus,mitochondrial 37S ribosomal protein VAR1,VAR1,854586,protein-coding,NaN,NP_009320.1,398.0,Q0140
6445,NC_001224.1,61022,61729,MT,plus,intron-encoded endonuclease I-SceI,SCEI,854590,protein-coding,NaN,NP_009324.1,235.0,Q0160
6462,NC_001224.1,73758,74513,MT,plus,cytochrome c oxidase subunit 2,COX2,854622,protein-coding,NaN,NP_009326.1,251.0,Q0250
6463,NC_001224.1,74495,75984,MT,plus,maturase-like protein,NaN,854623,protein-coding,NaN,NP_009327.1,472.0,Q0255


In [7]:
print(proteome["Locus tag"].isna().sum())

0


In [8]:
indices = {}
noms = {}
G = {}

for i, pid in enumerate(proteome["Locus tag"]):
    G[i+1] = set()
    indices[pid] = i+1
    noms[i+1] = pid

with open("Saccharomyces cerevisiae/interactome.txt", "r") as fil:
    fil.readline()
    for ligne in fil:
        v1, v2 = ligne[:-1].split(";")
        if (v1 in indices) and (v2 in indices):
            G[indices[v1]].add(indices[v2])
            G[indices[v2]].add(indices[v1])
        
nb_noeuds = len(G.keys())
nb_aretes = sum(list(map(len, list(G.values()))))/2

#G = {1: {2, 3}, 2: {1, 3}, 3: {1, 2, 4}, 4: {3, 5, 6}, 5: {4, 6}, 6: {5, 4}}

In [20]:
url = lambda pid: f"https://www.yeastgenome.org/backend/locus/{pid}/interaction_details"
reponse = requests.get(url("YAL068C")).json()
for interaction in reponse:
    if interaction["interaction_type"] == "Physical":
        print(interaction["locus1"], interaction["locus2"])

{'id': 1267611, 'display_name': 'PAU8', 'link': '/locus/S000002142', 'format_name': 'YAL068C'} {'id': 1285991, 'display_name': 'RCK1', 'link': '/locus/S000003126', 'format_name': 'YGL158W'}
{'id': 1281159, 'display_name': 'HEK2', 'link': '/locus/S000000128', 'format_name': 'YBL032W'} {'id': 1267611, 'display_name': 'PAU8', 'link': '/locus/S000002142', 'format_name': 'YAL068C'}
{'id': 1267611, 'display_name': 'PAU8', 'link': '/locus/S000002142', 'format_name': 'YAL068C'} {'id': 1283876, 'display_name': 'NAM7', 'link': '/locus/S000004685', 'format_name': 'YMR080C'}


In [26]:
with open("Saccharomyces cerevisiae/interactome_physique.txt", "w") as filout, open("Saccharomyces cerevisiae/non_reference_physique.txt", "w") as fil:
    texte, textref = "", ""
    for pid in proteome["Locus tag"]:
        reponse = requests.get(url(pid)).json()
        try:
            for interaction in reponse:
                if interaction["interaction_type"] == "Physical":
                    ss_texte = f"{interaction['locus1']['format_name']};{interaction['locus2']['format_name']}\n" if interaction['locus2']['format_name'] != pid else f"{interaction['locus2']['format_name']};{interaction['locus1']['format_name']}\n"
                    texte += ss_texte
        except:
            textref += f"{pid}\n"
    filout.write(texte)
    fil.write(textref)

YAL068C;YGL158W
YAL068C;YBL032W
YAL068C;YMR080C



In [27]:
with open("Saccharomyces cerevisiae/interactome_genetique.txt", "w") as filout, open("Saccharomyces cerevisiae/non_reference_genetique.txt", "w") as fil:
    texte, textref = "", ""
    for pid in proteome["Locus tag"]:
        reponse = requests.get(url(pid)).json()
        try:
            for interaction in reponse:
                if interaction["interaction_type"] == "Genetic":
                    ss_texte = f"{interaction['locus1']['format_name']};{interaction['locus2']['format_name']}\n" if interaction['locus2']['format_name'] != pid else f"{interaction['locus2']['format_name']};{interaction['locus1']['format_name']}\n"
                    texte += ss_texte
        except:
            textref += f"{pid}\n"
    filout.write(texte)
    fil.write(textref)

YAL068C;YDL225W
YAL068C;YMR044W
YAL068C;YDR054C
YAL068C;YDR404C
YAL068C;YJR065C
YAL068C;YMR235C
YAL068C;YNL061W
YAL068C;YOR074C
YAL068C;YOR204W
YAL068C;YOR236W
YAL068C;YLR142W
YAL068C;YBL020W
YAL068C;YBL034C
YAL068C;YDR531W
YAL068C;YFR005C
YAL068C;YGR277C
YAL068C;YJR045C
YAL068C;YDL140C
YAL068C;YDR311W
YAL068C;YDR510W
YAL068C;YER125W
YAL068C;YGL073W
YAL068C;YIL046W
YAL068C;YJL194W
YAL068C;YJL203W
YAL068C;YKL019W
YAL068C;YLR071C
YAL068C;YLR293C
YAL068C;YLR310C
YAL068C;YML130C
YAL068C;YOR020C
YAL068C;YOR174W
YAL068C;YPL082C
YAL068C;YPL128C
YAL068C;YBR017C
YAL068C;YDL225W
YAL068C;YDR022C
YAL068C;YER110C
YAL068C;YHL020C
YAL068C;YNCB0010W
YAL068C;YJL102W
YAL068C;YJR118C
YAL068C;YKL117W
YAL068C;YKR076W
YAL068C;YLR319C
YAL068C;YML020W
YAL068C;YML101C
YAL068C;YMR044W
YAL068C;YNL080C
YAL068C;YPL029W
YAL068C;YPL040C
YAL068C;YPR036W



In [32]:
with open("Saccharomyces cerevisiae/interactome.txt", "w") as filout, open("Saccharomyces cerevisiae/non_reference.txt", "w") as fil:
    texte, textref = "", ""
    for pid in proteome["Locus tag"]:
        reponse = requests.get(url(pid)).json()
        try:
            ss_texte = f"{pid};"
            ss_texte += f"{pid};".join(list(map(lambda x: x + "\n", [interaction['locus1']["format_name"] if interaction['locus1']["format_name"] != pid else interaction['locus2']["format_name"] for interaction in reponse])))
            texte += ss_texte
        except:
            textref += f"{pid}\n"
        break
    filout.write(texte)
    fil.write(textref)

55
